## Scoring

This notebook applies the scoring on the cleaned datasets and stores matched users.

Note: it was not used due to limited resources. Only subsets were taken into account, see notebook `scoring_subset`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dbb = pd.read_csv("time_fixed_DB.csv", encoding="UTF-8",sep=";")
dff= pd.read_csv("time_fixed_DF.csv", encoding="UTF-8",sep=";")

In [ ]:
db = dbb
df = dff.loc[(dff["userId"] > 100) & (dff["userId"] < 200)].copy()

In [ ]:
db.rename(columns={"userId":"uM"},inplace=1)
df.rename(columns={"userId":"uF"},inplace=1)
display(db.head())
display(df.head())

In [ ]:
weight = 1/np.log10(dff.loc[:,"movieId"].value_counts())
#del dff, dbb 

In [ ]:
db.set_index(["movieId"],inplace=True)
df.set_index(["movieId"],inplace=True)

In [ ]:
com_movies = df.index.unique().intersection(db.index.unique()) #Ordering, comment if working on all datasets
db = db.loc[com_movies,:]
df = df.loc[com_movies,:]
res = df.subtract(db,fill_value=0).astype(np.int64)
res = np.abs(res)

In [ ]:
result = weight.loc[com_movies]*(np.exp(-res.loc[:,'rating']/1.5) + np.exp(-res.loc[:,'timestamp']/30))

In [ ]:
res.loc[:,"score"] = result
res.drop(["rating", "timestamp"],axis=1,inplace=True)
res.set_index(['uF','uM'],inplace=True)
display(res.head())

In [ ]:
R = res.groupby(level=[0,1]).sum()

In [ ]:
display(R.head())
display(R.shape)

In [ ]:
values = R.index.levels[0]

for elem in values:
    tempUser = R.loc[elem]
    if elem == 164:
        plt.scatter(range(len(tempUser)),tempUser["score"])
        plt.show()
    tempScore = np.array(tempUser["score"])
    stdev = np.std(tempScore)
    best1 = np.max(tempScore)
    idx1 = np.argmax(tempScore)
    tempScore = np.delete(tempScore, idx1)
    # print(tempUser)
    best2 = np.max(tempScore)
    ecc = (best1-best2)/stdev
    print("User ID is", elem)
    if (ecc >= 1.5):
        print("Match with an eccentricity of", ecc)
    else:
        print("No match", ecc)
    print("\n")
    